# Week 11 - Introduction to Modeling, part 2

# 1. Lesson - No lesson this week

# 2. Weekly graph question

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt

The book names one of Vonnegut's rules as "keep it simple" and another as "have the guts to cut."  Here is some data from the previous week's lesson.  If you had to cut one of the two plots below, which would it be?  Which seems more interesting or important?  Explain.  (Should "amount of training data used" or "number of estimators" be on the x-axis.)

In [2]:
np.random.seed(0)
num_points = 10000
feature_1a = np.random.random(size = num_points) * 3
feature_2a = np.random.random(size = num_points) * 3
feature_3a = np.random.random(size = num_points) * 3
train_target = (feature_1a - 2 * feature_2a) * feature_3a + np.random.normal(size = num_points)
feature_1b = np.random.random(size = num_points) * 3
feature_2b = np.random.random(size = num_points) * 3
feature_3b = np.random.random(size = num_points) * 3
test_target = (feature_1b - 2 * feature_2b) * feature_3b + np.random.normal(size = num_points)
train_df = pd.DataFrame({"f1": feature_1a, "f2": feature_2a, "f3": feature_3a})
test_df = pd.DataFrame({"f1": feature_1b, "f2": feature_2b, "f3": feature_3b})
rf = RandomForestRegressor()
rf.fit(train_df.values, train_target)

In [3]:
rmse_lst = list()
rf = RandomForestRegressor()
for x in range(round(num_points / 20), num_points, round(num_points / 20)):
    rf.fit(train_df.values[0:x,:], train_target[0:x])
    rmse_lst.append(root_mean_squared_error(rf.predict(test_df.values), test_target))

import matplotlib.pyplot as plt

In [4]:
plt.plot(rmse_lst)
plt.xlabel("Amount of training data used (20 = max)")
plt.ylabel("Loss function")

In [5]:
num_trees_lst = list()
for n_estimators in range(1, 100, 3):
    rf = RandomForestRegressor(n_estimators = n_estimators)
    rf.fit(train_df.values, train_target)
    num_trees_lst.append(root_mean_squared_error(rf.predict(test_df.values), test_target))

In [6]:
plt.plot(num_trees_lst)
plt.xlabel("Number of estimators (trees)")
plt.ylabel("Loss function")

### If you had to cut one of the two plots below, which would it be?  Which seems more interesting or important?  Explain. (Should "amount of training data used" or "number of estimators" be on the x-axis.)
- The first plot shows RMSE as the size of the training set increases, with the RMSE dropping as the size of the training data is increased. As more data is added, the model "learns" to build a better representation of the underlying data and generalizes better, lowering the error. 
- The second plot shows the RMSE as the number of estimators in the decision tree model is increased, with the RMSE dropping as the complexity of the model is increased. As more trees are added, the model benefits from the ensemble effect, which will result in improved model as the model averages across many trees, improving accuracy and lowering error. 
- If I had to only pick one model, I'd go with the first plot because the learning curve (amount of training data used) offers a more fundamental insight into how the model improves with more data. Thus, having "amount of training data used" on the x-axis would be more interesting and important for understanding the overall behavior of the model.

# 3. Working on your datasets

This week, you will do the same types of exercises as last week, but you should use your chosen datasets that someone in your class found last semester. (They likely will not be the particular datasets that you found yourself.)

Here are some types of analysis you can do:

* Implement a random forest model.
* Perform cross-validation.
* Tune hyperparameters.
* Evaluate a performance metric.

If you like, you can try other types of models, too (beyond linear regression and random forest) although you will have many opportunities to do that next semester.

## Dataset 1

### Random Forest Classifier

In [7]:
# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    classification_report
)

# preprocess data
motion_df = pd.read_csv("datasets/nfl-playing-surface-analytics/motion_df_encoded.csv")

# Prepare the features (drop leakage columns)
X = motion_df.drop(columns=[
    'Injury', 'DM_M1', 'DM_M7', 'DM_M28', 'DM_M42'])

# One-hot encode categorical columns
X = pd.get_dummies(X, dummy_na=True, drop_first=True)

# For a cuDF DataFrame with some boolean columns:
for col in X.columns:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype('int32')


# Prepare the binary target (0 = no injury, 1 = injury)
y = motion_df['Injury'].copy()
y_binary = y.copy()
y_binary[y_binary > 0] = 1

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y_binary, 
    test_size=0.2,
    random_state=42,
    shuffle=True       
)

# Initialize and train a RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)

In [8]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
# 8. Predict and evaluate
y_pred = rf.predict(X_test)
y_pred_proba = rf.predict_proba(X_test)[:, 1]

# Classification report
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Confusion Matrix")
plt.show() # great accuracy but very poor recal

In [9]:
# 1. After rf.fit(X_train, y_train):
importances = rf.feature_importances_

# 2. Turn into a Series for easy sorting & plotting
feat_imp = pd.Series(importances, index=X_train.columns)

# 3. Sort descending and view the top 10
top10 = feat_imp.sort_values(ascending=False).head(10)
print(top10)

# 4. (Optional) Plot as a horizontal bar chart
plt.figure(figsize=(8,5))
top10.sort_values().plot.barh()
plt.title("Top 10 Feature Importances")
plt.xlabel("Relative Importance")
plt.tight_layout()
plt.show()

### Resampling with SMOTE, then Cross Validation through GridSearchCV

In [16]:
import time
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Start timing
start_time = time.time()

# Define the pipeline: first SMOTE, then the classifier.
pipeline = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=0))
])

# Parameter grid for grid search.
grid_params = {
    'classifier__max_depth': [2, 3, 4, 5, None],
    'classifier__min_samples_leaf': [1, 2, 3],
    'classifier__min_samples_split': [2, 3, 4],
    'classifier__max_features': [2, 3, 4],
    'classifier__n_estimators': [75, 100, 125, 150]
}

# Define the scoring metrics (ensure 'scoring' is defined appropriately).
scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

# Initialize GridSearchCV.
grid_cv = GridSearchCV(pipeline, 
                       grid_params, 
                       scoring=scoring, 
                       cv=3,
                       n_jobs=-1, 
                       refit='f1',
                       verbose=3)

# Fit on the training data.
grid_cv.fit(X_train, y_train)

# Print execution time.
execution_time = time.time() - start_time
print(f"\nExecution Time: {execution_time:.2f}s\n")

# Print the best parameters and best f1 score.
print("Best Parameters:")
print(grid_cv.best_params_)
print(f"\nBest F1 Score (CV): {grid_cv.best_score_:.4f}\n")

# Display all cross-validation results in a sorted DataFrame.
cv_results = pd.DataFrame(grid_cv.cv_results_)
# Sort by rank_test_f1 (lowest rank is the best)
cv_results = cv_results.sort_values('rank_test_f1')
print("Grid Search CV Results (top 5 rows):")
print(cv_results[['params', 'mean_test_f1', 'std_test_f1', 'rank_test_f1']].head())

# If you have a test set, evaluate on it:
if 'X_test' in globals() and 'y_test' in globals():
    y_pred = grid_cv.predict(X_test)
    print("\nClassification Report on Test Set:")
    print(classification_report(y_test, y_pred))
    
    print("Confusion Matrix on Test Set:")
    print(confusion_matrix(y_test, y_pred))

## Dataset 2

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import io
import zipfile
import time
from tqdm import tqdm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline

# Define the folder path
folder_path = 'datasets/nfl-big-data-bowl-2023.zip'
extract_path = 'datasets/nfl-big-data-bowl-2023'

# Check if the extraction directory already exists
if not os.path.exists(extract_path):
    # Unzip the file
    with zipfile.ZipFile(folder_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted all files to {extract_path}")
else:
    print(f"Directory {extract_path} already exists, skipping extraction.")

# List all CSV files in the folder
csv_files = [f for f in os.listdir(extract_path) if f.endswith('.csv')]

# Initialize an empty dictionary to store dataframes
dataframes = {}

# Load each CSV file as a separate dataframe with a progress bar
for file in tqdm(csv_files, desc="Loading CSV files", unit="file"):
    file_path = os.path.join(extract_path, file)
    df_name = os.path.splitext(file)[0]
    dataframes[df_name] = pd.read_csv(file_path)

# Convert dictionary to global variables
for key, df in dataframes.items():
    globals()[key] = df

# Display the keys of the dataframes dictionary to verify
print("Loaded DataFrames:", list(dataframes.keys()))

# concat all motion data
all_weeks = pd.concat([week1, week2, week3, week4, week5, week6, week7, week8], ignore_index=True)
all_weeks = all_weeks.merge(players[['nflId', 'displayName', 'officialPosition']], on='nflId', how='left')

# group all motion data by gameId, playId, nflId, displayName, officialPosition, then calculate mean, max, std of s, a, x, y
motion_df = all_weeks.groupby(['gameId', 'playId', 'nflId', 'displayName', 'officialPosition']).agg({
    's': ['mean', 'max', 'std'],
    'a': ['mean', 'max', 'std'],
    'x': ['min', 'max', 'mean'],
    'y': ['min', 'max', 'mean']
})

# flatten the multi-level columns
motion_df = motion_df.reset_index()
motion_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in motion_df.columns.values]

# Merge pffScoutingData into motion_df
motion_df = motion_df.merge(pffScoutingData, on=['gameId', 'playId', 'nflId'], how='left')
motion_df = motion_df.merge(plays, on=['gameId', 'playId'], how='left')
ol = ['C', 'G', 'T', 'TE']
dl = ['DL', 'LB', 'DE', 'OLB', 'ILB', 'NT', 'MLB']
motion_df['OL'] = motion_df['officialPosition'].apply(lambda x: 1 if x in ol else 0)
motion_df['DL'] = motion_df['officialPosition'].apply(lambda x: 1 if x in dl else 0)

#create separate dataframes for OL and DL
ol_motion_df = motion_df[motion_df['OL'] == 1].copy()
dl_motion_df = motion_df[motion_df['DL'] == 1].copy()


# set up dataset
feature_cols = ['s_mean', 's_max', 's_std', 'a_mean', 'a_max', 'a_std', 'x_min',
       'x_max', 'x_mean', 'y_min', 'y_max', 'y_mean', 'defendersInBox', 'absoluteYardlineNumber',
       'pff_beatenByDefender', 'pff_hitAllowed', 'pff_hurryAllowed',
       'pff_sackAllowed', 'pff_blockType', 'pff_backFieldBlock', 'playResult']

motion_model = motion_df[feature_cols].copy()
motion_model = motion_model.dropna(subset=['pff_blockType', 'pff_backFieldBlock', 'defendersInBox'])

X = motion_model.drop(columns=['playResult'])
y = motion_model['playResult']

X = pd.get_dummies(X, columns=['pff_blockType'], drop_first=True, dtype='int')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest R^2 score: {r2_rf:.4f}")
print(f"Random Forest Mean Squared Error: {mse_rf:.4f}")
# Get feature importances
feature_importance_rf = rf.feature_importances_
importance_df_rf = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importance_rf
}).sort_values(by='importance', ascending=False)
print(importance_df_rf)

In [ ]:
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)

# Parameter grid for RandomForestRegressor tuning.
param_tests_rf = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [0.5, 0.7, 1.0],
    'bootstrap': [True, False],
    'warm_start': [False, True]
}

# Record the start time.
start_time = time.time()

# Instantiate the RandomForestRegressor with a fixed random state.
rf_model = RandomForestRegressor(random_state=42)

# Set up cross-validation: here we use 5-fold cross-validation repeated 3 times.
cv = RepeatedKFold(n_splits=3, n_repeats=1, random_state=42)

# Set up the RandomizedSearchCV.
# n_iter controls the number of random combinations to try.
random_search_rf = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_tests_rf,
    n_iter=100,                          # Adjust n_iter for more/less thorough search.
    scoring='neg_mean_squared_error',   # We use negative MSE; will convert later.
    cv=cv,
    n_jobs=12,
    verbose=1,                          # Verbose prints progress.
    random_state=42
)

# Fit the model using the reduced training data for RandomForest.
random_search_rf.fit(X_train, y_train)

# Extract the results into a DataFrame.
results_rf_df = pd.DataFrame(random_search_rf.cv_results_)

# Convert negative MSE back to positive MSE and then compute RMSE.
results_rf_df['mean_test_MSE'] = -results_rf_df['mean_test_score']
results_rf_df['mean_test_RMSE'] = np.sqrt(results_rf_df['mean_test_MSE'])

# Sort the results by RMSE (lowest is best).
sorted_results_rf = results_rf_df.sort_values(by='mean_test_RMSE')

# Display the top 10 parameter combinations (by RMSE).
print("\nTop 10 Parameter Combinations by RMSE:")
print(sorted_results_rf[['param_n_estimators', 
                           'param_max_depth', 
                           'param_min_samples_split',
                           'param_min_samples_leaf', 
                           'param_max_features', 
                           'param_bootstrap',
                           'param_warm_start',
                           'mean_test_RMSE']].head(10))

# Get and print the best parameters and corresponding RMSE.
best_params_rf = random_search_rf.best_params_
best_rmse_rf = np.sqrt(-random_search_rf.best_score_)

print(f"\nBest Parameters: {best_params_rf}")
print(f"Best RMSE: {best_rmse_rf:.4f}")

# Display the execution time.
execution_time_rf = time.time() - start_time
print(f"\nExecution Time: {execution_time_rf:.2f}s")

## Dataset 3

In [13]:
import pandas as pd
import numpy as np

def load_and_normalize(path):
    """Load a CSV, strip and lowercase its column names."""
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip().str.lower()
    return df

# 1. LOAD & NORMALIZE
# -------------------
games            = load_and_normalize('datasets/NFL-Punt-Analytics-Competition/game_data.csv')
play_info        = load_and_normalize('datasets/NFL-Punt-Analytics-Competition/play_information.csv')
player_roles     = load_and_normalize('datasets/NFL-Punt-Analytics-Competition/play_player_role_data.csv')
player_positions = load_and_normalize('datasets/NFL-Punt-Analytics-Competition/player_punt_data.csv')
video_review     = load_and_normalize('datasets/NFL-Punt-Analytics-Competition/video_review.csv')

# NGS chunks for 2016 & 2017
ngs_paths = [
    'datasets/NFL-Punt-Analytics-Competition/NGS-2016-pre.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2016-post.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk1-6.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk7-12.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2016-reg-wk13-17.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2017-pre.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2017-post.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2017-reg-wk1-6.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2017-reg-wk7-12.csv',
    'datasets/NFL-Punt-Analytics-Competition/NGS-2017-reg-wk13-17.csv',
]
ngs = pd.concat([load_and_normalize(p) for p in ngs_paths], ignore_index=True)


# 2. CLEAN & CAST
# ----------------

# Parse any ISO‐style dates in games & play_info
for df in (games, play_info):
    if 'game_date' in df.columns:
        df['game_date'] = pd.to_datetime(df['game_date'])

# Make sure keys are ints
for df in (player_roles, player_positions, video_review):
    for col in ('gamekey', 'playid', 'gsisid'):
        if col in df.columns:
            df[col] = df[col].astype(int)

# Parse NGS timestamps if present
if 'time' in ngs.columns:
    ngs['time'] = pd.to_datetime(ngs['time'])


# 3. MERGE BASE TABLE
# -------------------

# Start from each player’s role in each play
df = player_roles.copy()

# Merge play-level data
df = df.merge(
    play_info,
    on=['gamekey','playid'],
    how='left',
    validate='many_to_one'
)

# Merge game-level data
df = df.merge(
    games.drop(columns=['game_date'], errors='ignore'),
    on='gamekey',
    how='left',
    validate='many_to_one'
)

# Merge typical football position
if {'gamekey','gsisid','position'}.issubset(player_positions.columns):
    df = df.merge(
        player_positions[['gamekey','gsisid','position']],
        on=['gamekey','gsisid'],
        how='left'
    )


# 4. ADD VIDEO REVIEW AS FEATURES
# --------------------------------

# Select the video_review columns we want
video_feats = video_review[[
    'gamekey','playid','gsisid',
    'player_activity_derived',
    'turnover_related',
    'primary_impact_type',
    'primary_partner_activity_derived',
    'friendly_fire'
]]

# Merge them in
df = df.merge(
    video_feats,
    on=['gamekey','playid','gsisid'],
    how='left'
)

# Fill NaNs for non-injured rows
for c in [
    'player_activity_derived',
    'turnover_related',
    'primary_impact_type',
    'primary_partner_activity_derived',
    'friendly_fire'
]:
    df[c] = df[c].fillna('NoInjury')


# 5. BUILD TARGET—ANY INJURY
# --------------------------

inj = video_review[['gamekey','playid','gsisid']].copy()
inj['injury'] = 1

df = df.merge(inj, on=['gamekey','playid','gsisid'], how='left')
df['injury'] = df['injury'].fillna(0).astype(int)


# 6. AGGREGATE NGS INTO SUMMARY FEATURES
# ---------------------------------------

if {'gamekey','playid','gsisid','dis'}.issubset(ngs.columns):
    ngs_summary = (
        ngs
        .groupby(['gamekey','playid','gsisid'], as_index=False)
        .agg(
            total_distance = ('dis','sum'),
            max_step       = ('dis','max'),
            mean_step      = ('dis','mean'),
            n_timestamps   = ('time' if 'time' in ngs.columns else 'dis','count')
        )
    )
    df = df.merge(ngs_summary, on=['gamekey','playid','gsisid'], how='left')
else:
    df[['total_distance','max_step','mean_step','n_timestamps']] = 0


# 7. FINAL PREP FOR EDA & MODELING
# ---------------------------------

# Fill any remaining nulls in the summary stats
for col in ['total_distance','max_step','mean_step','n_timestamps']:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Define feature columns
feature_cols = [
    'position', 'role', 'season_year', 'week',
    'player_activity_derived',
    'turnover_related',
    'primary_impact_type',
    'primary_partner_activity_derived',
    'friendly_fire',
    'total_distance','max_step','mean_step','n_timestamps'
]
feature_cols = [c for c in feature_cols if c in df.columns]

# One‑hot encode categoricals
X = pd.get_dummies(df[feature_cols], drop_first=True)
y = df['injury']

# Quick check
print("▶️  Data prep complete!")
print("   X shape:", X.shape)
print("   Injury prevalence:", y.mean())

In [14]:
mapping = {
  'PR':'Returner',
  'GL':'Gunner','GR':'Gunner',
  'PLW':'Wing','PLG':'Wing','PRG':'Wing','PLL':'Wing',
  # …etc…
}
df['role_group'] = df['role'].map(mapping)

# 2) Drop all the old one‑hot role_ columns
old_roles = [c for c in X.columns if c.startswith('role_')]
X = X.drop(columns=old_roles)

# 3) Add one‑hot encoding for the three groups
role_group_dummies = pd.get_dummies(df['role_group'], prefix='role_group', drop_first=True)
X = pd.concat([X.reset_index(drop=True), role_group_dummies.reset_index(drop=True)], axis=1)

print("New feature set:")
print(X.filter(like='role_group_').columns)

In [15]:
# Identify and drop all columns that came from video_review
leak_cols = [c for c in X.columns if 
             c.startswith('player_activity_derived_') or
             c.startswith('turnover_related_') or
             c.startswith('primary_impact_type_') or
             c.startswith('primary_partner_activity_derived_') or
             c.startswith('friendly_fire_')]

X_nomap = X.drop(columns=leak_cols)

In [16]:
# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    classification_report
)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    X_nomap, y, test_size=0.25, stratify=y, random_state=42
)

scaler = StandardScaler().fit(X_train)
X_tr = scaler.transform(X_train)
X_te = scaler.transform(X_test)

# Initialize and train a RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=10, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

# Predict and evaluate
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)

In [17]:
# 2. Pull out the importances and map to feature names
importances = rf.feature_importances_
feat_imp = pd.Series(importances, index=X_train.columns)

# 3. Sort and display the top 10
top10 = feat_imp.sort_values(ascending=False).head(10)
print(top10)

# 4. (Optional) Plot as a horizontal bar chart
plt.figure(figsize=(8, 5))
top10.sort_values().plot.barh()
plt.title("Top 10 Feature Importances")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()

## Model Rerun with Class Balancing

In [18]:
# random forest with SMOTE
from imblearn.over_sampling import SMOTE

# e.g. make injuries 50% of the training set
smote = SMOTE(sampling_strategy=0.5, random_state=42)  

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Initialize and train a RandomForestClassifier
rf_smote = RandomForestClassifier(n_estimators=100, max_depth=10, class_weight ='balanced_subsample', random_state=42)
rf_smote.fit(X_train_resampled, y_train_resampled)

# Predict and evaluate
y_pred = rf_smote.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)


# 8. Predict and evaluate
y_pred = rf_smote.predict(X_test)
y_pred_proba = rf_smote.predict_proba(X_test)[:, 1]

# Classification report
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Confusion Matrix")
plt.show()

## GridsearchCV with SMOTE

In [ ]:
import time
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Start timing
start_time = time.time()

# Define the pipeline: first SMOTE, then the classifier.
pipeline = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(random_state=0))
])

# Parameter grid for grid search.
grid_params = {
    'classifier__max_depth': [2, 3, 4, 5, None],
    'classifier__min_samples_leaf': [1, 2, 3],
    'classifier__min_samples_split': [2, 3, 4],
    'classifier__max_features': [2, 3, 4],
    'classifier__n_estimators': [75, 100, 125, 150]
}

# Define the scoring metrics (ensure 'scoring' is defined appropriately).
scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

# Initialize GridSearchCV.
grid_cv = GridSearchCV(pipeline, 
                       grid_params, 
                       scoring=scoring, 
                       cv=3,
                       n_jobs=-1, 
                       refit='f1',
                       verbose=2)

# Fit on the training data.
grid_cv.fit(X_train, y_train)

# Print execution time.
execution_time = time.time() - start_time
print(f"\nExecution Time: {execution_time:.2f}s\n")

# Print the best parameters and best f1 score.
print("Best Parameters:")
print(grid_cv.best_params_)
print(f"\nBest F1 Score (CV): {grid_cv.best_score_:.4f}\n")

# Display all cross-validation results in a sorted DataFrame.
cv_results = pd.DataFrame(grid_cv.cv_results_)
# Sort by rank_test_f1 (lowest rank is the best)
cv_results = cv_results.sort_values('rank_test_f1')
print("Grid Search CV Results (top 5 rows):")
print(cv_results[['params', 'mean_test_f1', 'std_test_f1', 'rank_test_f1']].head())

# If you have a test set, evaluate on it:
if 'X_test' in globals() and 'y_test' in globals():
    y_pred = grid_cv.predict(X_test)
    print("\nClassification Report on Test Set:")
    print(classification_report(y_test, y_pred))
    
    print("Confusion Matrix on Test Set:")
    print(confusion_matrix(y_test, y_pred))

# 4. Storytelling With Data plot

Reproduce any graph of your choice in chapter seven (p. 165-185) of the Storytelling With Data book as best you can. You do not have to get the exact data values right, just the overall look and feel.

PS Note: No graphs in the indicated Section, I picked one from chapter 8 instead.

In [21]:
import matplotlib.pyplot as plt
import numpy as np


products = ['Product A', 'Product B', 'Product C', 'Product D', 'Product E']

product_years = {
    'Product A': [2008, 2010, 2014],
    'Product B': [2008, 2010, 2014],
    'Product C': [2010, 2012, 2014],
    'Product D': [2011, 2012.5, 2014],
    'Product E': [2013, 2013.5, 2014]
}

product_prices = {
    'Product A': [420, 410, 280],
    'Product B': [400, 390, 260],
    'Product C': [100, 230, 200],
    'Product D': [150, 250, 210],
    'Product E': [90, 200, 220]
}

# Plot setup
plt.figure(figsize=(10, 5))
for idx, product in enumerate(products):
    # Offset x values to group by product index
    x_vals = np.linspace(idx, idx + 0.4, len(product_years[product]))
    y_vals = product_prices[product]
    years = product_years[product]

    # Draw the line
    plt.plot(x_vals, y_vals, color='gray', linewidth=5, alpha=0.8)

    # Annotate beginning and end years
    plt.text(x_vals[0], y_vals[0]+10, str(years[0]), color='gray', ha='center')
    plt.text(x_vals[-1], y_vals[-1]-20, str(years[-1]), color='gray', ha='center')

plt.xticks(range(len(products)), products)
plt.yticks(range(0, 501, 100), [f"${v}" for v in range(0, 501, 100)])
plt.ylim(0, 500)
plt.title("Average Retail Product Price per Year", fontsize=14, weight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.4)
plt.box(True)

plt.tight_layout()
plt.show()